In [1]:
#import libraries
import pandas as pd
import numpy as np
import nltk
#from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import demoji
import pickle
import re
from sklearn.model_selection import train_test_split

In [2]:
nltk.download('punkt')

In [3]:
%%time
tweets_complete = pd.read_table('conv_all.txt',sep=',')

In [4]:
tweets_complete.shape

In [5]:
tweets_complete[0:2]

## Data Preprocessing

In [6]:
tweets = tweets_complete

In [7]:
with open(r'C:\Users\AishwaryaBhangale\Documents\GitHub\nlp-coe\twitter-conversational-chatbot\Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        text = re.sub(r'(' +emot+ ')', "".join(Emoji_Dict[emot].replace(":"," ")),text)
    return text

def remove_URL(text):
    return re.sub(r"http\S+", "HTTPS_URL", str(text))

def sn_replace(match):
    _sn = match.group(2).lower()
    if not _sn.isnumeric():
        # This is a company screen name
        return match.group(1) + match.group(2)
    return ' @__sn__'



In [ ]:
#Define stop words
#stop_words = set(stopwords.words('english'))
#define special characters list
#special_characters = ['!','#','$','%', '&','@','[',']',' ',']','_']

tweets['text_x_clean'] = ""
tweets['text_y_clean'] = ""

for i in range(len(tweets)):

    #################################################################################################################
    
    
    
    tweets['text_x_clean'][i] = tweets['text_x'][i].lower()
    
    #Conversion of Emojis to Text
    tweets['text_x_clean'][i] = convert_emojis_to_word(tweets['text_x_clean'][i])
    
    #Removal of URLS
    pattern=r'(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))';
    match = re.findall(pattern, tweets['text_x_clean'][i])
    for m in match:
        url = m[0]
        tweets['text_x_clean'][i] = tweets['text_x_clean'][i].replace(url, 'URL')
    
    #Removal of HTTPS with HTTPS_URL
    tweets['text_x_clean'][i] = remove_URL(tweets['text_x_clean'][i])
    
    #Replacing with _sn_
    sn_re = re.compile('(\W@|^@)([a-zA-Z0-9_]+)')
    tweets['text_x_clean'][i] =  sn_re.sub(sn_replace, tweets['text_x_clean'][i])
    
    #print(tweets['text_x_clean'][i])
    #################################################################################################################
    
    tweets['text_y_clean'][i] = tweets['text_y'][i].lower()
    
    #Conversion of Emojis to Text
    tweets['text_y_clean'][i] = convert_emojis_to_word(tweets['text_y_clean'][i])
    
    #Replacement of url string with keyword URL
    pattern=r'(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))';
    match = re.findall(pattern, tweets['text_y_clean'][i])
    for m in match:
        url = m[0]
        tweets['text_y_clean'][i] = tweets['text_y_clean'][i].replace(url, 'URL')
    
    #Replacement of HTTPS
    tweets['text_y_clean'][i] = remove_URL(tweets['text_y_clean'][i])

    #Replacing with _sn_
    sn_re = re.compile('(\W@|^@)([a-zA-Z0-9_]+)')
    tweets['text_y_clean'][i] =  sn_re.sub(sn_replace, tweets['text_y_clean'][i])    
    
    #print(tweets['text_y_clean'][i])

    #################################################################################################################
    

In [ ]:
#Split into train and test
x_train, x_test, y_train, y_test = train_test_split(tweets['text_x_clean'], tweets['text_y_clean'], test_size=0.25, random_state=11)


In [ ]:
x_train.head()

In [ ]:
x_train.shape

In [ ]:
x_test.shape